In [4]:
from os.path import join
import os 
import glob
import datetime
import pandas as pd
from update import *

class forex():

    def __init__(self, pair, path="history", mode="min", dtype=None, quiet=True):
        

        self.path_pair = join(path,pair)
        self.pair = pair
        self.mode = mode
    
        if (not os.path.exists(join(path))): # Creates core history
            os.mkdir(path) 

        if (self.path_pair not in glob.glob(join(path,"*"))): # Creates Pair dir
            os.mkdir(self.path_pair) 

        if (not os.path.exists(join(self.path_pair, f"{pair}_update.txt"))): # Creates update.txt
            temp = open(join(self.path_pair,f"{pair}_update.txt"), 'x')
            temp.write("0")
            temp.close()
        
        if (join(self.path_pair, "news") not in glob.glob(join(self.path_pair, "news"))): # Creates news
            os.mkdir(join(self.path_pair, "news"))

        g = open(join(self.path_pair, f"{self.pair}_update.txt"), "r+")
        temp = g.readlines()[-1]
        
        # Daily updates
        if (temp != datetime.datetime.today().strftime("%m-%d-%Y")):
            print("Note this may take a while")
            print("Updating...")
            update_pair(self.pair)
            # update_news_raw(self.pair, path = self.path)
            g.write("\n" + datetime.datetime.today().strftime("%m-%d-%Y"))
        g.close()



        self.hr_const = pd.read_csv(join(self.path_pair, "hr.csv"))
        self.min_const = pd.read_csv(join(self.path_pair, "min.csv"))
        self.day_const = pd.read_csv(join(self.path_pair,"day.csv"))

        self.hr = pd.read_csv(join(self.path_pair, "hr.csv"))
        self.min = pd.read_csv(join(self.path_pair, "min.csv"))
        self.day = pd.read_csv(join(self.path_pair, "day.csv"))

    def update(self,force_news = False):
        update_pair(self.pair)
        if force_news:
            update_labels(self.pair) 
    
    def add_senti(self,force_update=False):
      # Adds sentiment column 

        # Just checks if the days need updating based on len
        aday = pd.read_csv(join(self.path_pair, "news_day.csv"))
        cday = pd.read_csv(join(self.path_pair ,"day.csv"))


        if (len(aday) != len(cday) or force_update):
            update_labels(self.pair)

        self.day = self.day_const
        self.hr  = self.hr_const
        self.min = self.min_const


        aday = pd.read_csv(join(self.path_pair, "news_day.csv"))
        ahr = pd.read_csv(join(self.path_pair,"news_hr.csv"))
        amin = pd.read_csv(join(self.path_pair,"news_min.csv"))
        
        self.day = pd.concat([self.day,aday],axis=1)
        self.hr = pd.concat([self.hr,ahr],axis=1)
        self.min = pd.concat([self.min,amin],axis=1)
